In [4]:
train_path = '/kaggle/input/cards-image-datasetclassification/train'
test_path = '/kaggle/input/cards-image-datasetclassification/test'

In [5]:
import torch
from pathlib import Path
from torch import nn
from torch import optim
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,random_split
from torchvision import transforms, datasets
from torchmetrics import Accuracy
from torch.utils.data import DataLoader,Subset
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [7]:
train_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomRotation(degrees=10),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])
basic_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])

In [8]:
train_dataset = datasets.ImageFolder(train_path,transform=train_transforms)

In [9]:
test_dataset = datasets.ImageFolder(test_path,transform=basic_transforms)

In [10]:
class_names = train_dataset.classes
len(class_names)

53

In [11]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 7624
    Root location: /kaggle/input/cards-image-datasetclassification/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
               ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [12]:
train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=64,shuffle=True)

In [13]:
def AlexNet(num_classes=53):
  def _initialize_weights(m):
        if type(m) == torch.nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            torch.nn.init.constant_(m.bias, 0.0)
        if type(m) == torch.nn.Conv2d:
            torch.nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
              torch.nn.init.constant_(m.bias, 0.0)

  net = nn.Sequential(
        nn.Conv2d(3, 96, kernel_size=11, stride=4,padding=2),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3,stride=2),

        nn.Conv2d(96, 256, kernel_size=5, padding=2),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3,stride=2),

        nn.Conv2d(256, 384, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),

        nn.Conv2d(384, 384, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),

        nn.Conv2d(384, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.AdaptiveAvgPool2d((6, 6)),
        nn.Flatten(),

        nn.Linear(256 * 6 * 6, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),

        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),
        nn.Linear(4096, num_classes))
  net.apply(_initialize_weights)

  return net

# Создание модели
model = AlexNet(num_classes=len(class_names))

In [14]:
from sklearn.metrics import classification_report

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.9,weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

def simple_accuracy(preds, targets):
    return (preds == targets).float().mean()
model.to(device)

Sequential(
  (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (13): AdaptiveAvgPool2d(output_size=(6, 6))
  (14): Flatten(start_dim=1, end_dim=-1)
  (15): Linear(in_features=9216, out_features=4096, bias=True)
  (16): ReLU(inplace=True)
  (17): Dropout(p=0.5, inplace=False)
  (18): Linear(in_fea

In [16]:
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
#Тренирует
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_targets = []

    progress_bar = tqdm(dataloader, desc='Training')
    for batch_idx, (data, target) in enumerate(progress_bar):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        all_preds.append(predicted)
        all_targets.append(target)
        progress_bar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'Avg Loss': f'{running_loss/(batch_idx+1):.4f}'
        })

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = simple_accuracy(torch.cat(all_preds), torch.cat(all_targets))

    return epoch_loss, epoch_acc
#Тестит
def evaluate(model, dataloader, criterion, device,matrix=0):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data, target in tqdm(dataloader, desc='Evaluating'):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)

            running_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            all_preds.append(predicted.cpu())
            all_targets.append(target.cpu())

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = simple_accuracy(torch.cat(all_preds), torch.cat(all_targets))
    if matrix != 0:
        all_preds2 = np.concatenate(all_preds)
        all_targets2 = np.concatenate(all_targets)
        print()
        print(classification_report(all_targets2, all_preds2, target_names=class_names))
    return epoch_loss, epoch_acc


epochs = 15


for epoch in range(epochs):
    print(f"\nЭпоха {epoch+1}/{epochs}")
    print("-" * 50)

    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc.cpu())

    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc.cpu())

    scheduler.step()

    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")



Эпоха 1/15
--------------------------------------------------


Evaluating: 100%|██████████| 5/5 [00:08<00:00,  1.70s/it]


Train Loss: 3.8875, Train Acc: 0.0317
Test Loss: 3.6018, Test Acc: 0.0943

Эпоха 2/15
--------------------------------------------------


Evaluating: 100%|██████████| 5/5 [00:06<00:00,  1.31s/it]


Train Loss: 3.2402, Train Acc: 0.1229
Test Loss: 3.3423, Test Acc: 0.1208

Эпоха 3/15
--------------------------------------------------


...

Train Loss: 1.1974, Train Acc: 0.6672
Test Loss: 0.7108, Test Acc: 0.8000

Эпоха 14/15
--------------------------------------------------


Evaluating: 100%|██████████| 5/5 [00:06<00:00,  1.30s/it]


Train Loss: 1.1379, Train Acc: 0.6817
Test Loss: 0.8145, Test Acc: 0.8151

Эпоха 15/15
--------------------------------------------------


Evaluating: 100%|██████████| 5/5 [00:06<00:00,  1.33s/it]

Train Loss: 1.1179, Train Acc: 0.6857
Test Loss: 0.7735, Test Acc: 0.7925


In [17]:
final_test_loss, final_test_acc = evaluate(model, test_loader, criterion, device, 1)
     

Evaluating: 100%|██████████| 5/5 [00:06<00:00,  1.28s/it]


                   precision    recall  f1-score   support

     ace of clubs       0.42      1.00      0.59         5
  ace of diamonds       1.00      1.00      1.00         5
    ace of hearts       0.56      1.00      0.71         5
    ace of spades       0.83      1.00      0.91         5
   eight of clubs       1.00      1.00      1.00         5
eight of diamonds       0.83      1.00      0.91         5
  eight of hearts       1.00      1.00      1.00         5
  eight of spades       1.00      0.80      0.89         5
    five of clubs       0.75      0.60      0.67         5
 five of diamonds       1.00      0.80      0.89         5
   five of hearts       1.00      1.00      1.00         5
   five of spades       1.00      1.00      1.00         5
    four of clubs       0.80      0.80      0.80         5
 four of diamonds       0.83      1.00      0.91         5
   four of hearts       0.71      1.00      0.83         5
   four of spades       0.75      0.60      0.67      